Iniciando Sessão SPARK

spark.default.parallelism:
Este parâmetro define o número padrão de partições a serem criadas ao ler arquivos e ao executar operações como mapas em RDDs. Definir isso para um valor mais alto pode aumentar o número de tarefas.

spark.sql.shuffle.partitions:
Esse parâmetro define o número de partições a serem usadas ao embaralhar dados para operações de junção, agregação, etc., em DataFrames. Aumentar esse valor pode aumentar o número de tarefas geradas durante essas operações.

spark.executor.instances e spark.executor.cores:
Estes parâmetros controlam o número de executores e o número de núcleos por executor, respectivamente. Aumentar o número de executores e/ou o número de núcleos por executor pode aumentar o número total de tarefas que podem ser executadas simultaneamente.

spark.task.cpus:
Este parâmetro define o número máximo de núcleos de CPU a serem usados ​​por tarefa. Aumentar esse valor pode permitir que mais tarefas sejam executadas simultaneamente em um único executor.


spark = SparkSession.builder \
    .appName("MyApp") \
    .config("spark.default.parallelism", "100") \
    .config("spark.sql.shuffle.partitions", "200") \
    .config("spark.executor.instances", "4") \
    .config("spark.executor.cores", "4") \
    .config("spark.task.cpus", "1") \
    .config("spark.ui.retainedTasks", "10") \
    .getOrCreate()

In [20]:
!pip install pymongo
!pip install findspark
!pip install python-dotenv


[notice] A new release of pip is available: 24.3.1 -> 26.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 26.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 26.0.1
[notice] To update, run: pip install --upgrade pip


In [21]:

from pyspark.sql import SparkSession
import findspark
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()


os.environ["spark_home"] = "\\spark-3.1.2-bin-hadoop2.7"
findspark.init()
spark = SparkSession.builder \
    .master('local[*]') \
    .appName("Iniciando com Spark") \
    .config("spark.default.parallelism", "100") \
    .config("spark.executor.instances", "10") \
    .config("spark.executor.cores", "10") \
    .getOrCreate()
spark

**Importando Lista de IPs**

In [22]:
# Importe a biblioteca requests
import requests

# Faça uma solicitação HTTP para obter o conteúdo do arquivo
banlist = "http://cinsscore.com/list/ci-badguys.txt"
response = requests.get(banlist)
lines = response.text.splitlines()
lines = lines[:100] #pegar somente 10 IPs

# Cria um RDD (Resilient Distributed Dataset) chamando ips a partir das linhas fornecidas,
# distribuindo essas linhas entre os nós do cluster Spark
# para processamento paralelo. Este RDD pode então ser usado
# para realizar operações distribuídas em larga escala nos dados.
ips = spark.sparkContext.parallelize(lines)

**Função para acessar a API para enriquecer dados**

In [23]:
#Função para acrescentar dados ao IP
import requests

#IP INFO
def get_ip_info(ip):
    IPINFO_AK = os.getenv("IPINFO_AK")

    url = "https://ipinfo.io/{}/json"
    headers = {
        "Authorization": f"Bearer {IPINFO_AK}"
    }
    response = requests.get(url.format(ip), headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Erro {response.status_code}: {response.text}")
        return {}

#ABUSE IP PD
def abuseipdb(ip):
    url = 'https://api.abuseipdb.com/api/v2/check'
    API_KEY = os.getenv("ABUSE_AK")

    querystring = {
        'ipAddress': ip,
        'maxAgeInDays': '90'  # Período de tempo para considerar relatórios (em dias)
    }
    headers = {
        'Accept': 'application/json',
        'Key': API_KEY
    }

    response = requests.get(url, headers=headers, params=querystring)
    if response.status_code == 200:
        return response.json()
    else:
        return {}

#print(abuseipdb('1.14.3.240'))
#print(get_ip_info('1.14.3.240'))

**Script para enriquecer dados**

In [24]:
import json
import datetime as dt

# Pegando a quantidade de IPs importados
num_ips = ips.count()
print("Número total de IPs:", num_ips)

if num_ips > 0:
    inicio_tarefa = dt.datetime.now()

    # Coleta as informações para todos os IPs de uma só vez
    ip_infos = {ip: get_ip_info(ip.strip()) for ip in ips.collect()}

    # Coleta os dados do abuseipdb para todos os IPs de uma só vez
    abuse_infos = {ip: abuseipdb(ip) for ip in ips.collect()}

    # Map para enriquecer os dados
    enriched_data_rdd = ips.map(lambda ip: {
        'listIP': ip,
        'listName': 'cinsscore',
        'RecommendedUsage': 'Incoming',
        'Description': 'cinsscore all list contains IP addresses.',
        'Timestamp': dt.datetime.now().isoformat(),

        # Informações do get_ip_info
        'city': ip_infos[ip].get('city', ''),
        'region': ip_infos[ip].get('region', ''),
        'country': ip_infos[ip].get('country', ''),
        'latitude': ip_infos[ip].get('loc', '').split(',')[0],
        'longitude': ip_infos[ip].get('loc', '').split(',')[1],
        'org': ip_infos[ip].get('org', ''),
        'postal': ip_infos[ip].get('postal', ''),
        'timezone': ip_infos[ip].get('timezone', ''),

        # Informações do abuseipdb
        'isPublic': abuse_infos[ip].get('data', {}).get('isPublic', ''),
        'ipVersion': abuse_infos[ip].get('data', {}).get('ipVersion', ''),
        'isWhitelisted': abuse_infos[ip].get('data', {}).get('isWhitelisted', 'False'),
        'abuseConfidenceScore': abuse_infos[ip].get('data', {}).get('abuseConfidenceScore', ''),
        'usageType': abuse_infos[ip].get('data', {}).get('usageType', ''),
        'isp': abuse_infos[ip].get('data', {}).get('isp', ''),
        'domain': abuse_infos[ip].get('data', {}).get('domain', ''),
        'totalReports': abuse_infos[ip].get('data', {}).get('totalReports', 0),
        'lastReportedAt': abuse_infos[ip].get('data', {}).get('lastReportedAt', ''),
        'numDistinctUsers': abuse_infos[ip].get('data', {}).get('numDistinctUsers', 0)
    })

    fim_tarefa = dt.datetime.now()
    print('Duração enriquecimento:', fim_tarefa - inicio_tarefa)

Número total de IPs: 100
Duração enriquecimento: 0:01:20.055399


 **Carga no MongoDB (UPSERT)**

In [29]:
from pymongo import MongoClient
import datetime as dt

# --- Coletar os resultados do RDD localmente ---
enriched_data = enriched_data_rdd.collect()

print(f"📦 Registros a inserir: {len(enriched_data)}")

if not enriched_data:
    raise Exception("Nenhum dado para inserir no MongoDB")

# --- Conexão MongoDB ---
MONGO_URI = os.environ.get("MONGO_URI")
DB_NAME = "etl_ips"
COLLECTION_NAME = "cinsscore_ips"

client = MongoClient(MONGO_URI)
db = client[DB_NAME]
collection = db[COLLECTION_NAME]

# (Opcional) limpar coleção antes de inserir tudo novamente
# collection.delete_many({})
# print("🗑️ Coleção limpa")

inicio_tarefa = dt.datetime.now()

# --- INSERÇÃO EM MASSA ---
result = collection.insert_many(enriched_data)

fim_tarefa = dt.datetime.now()

print("⏱️ Duração ingestão MongoDB:", fim_tarefa - inicio_tarefa)
print(f"✅ {len(result.inserted_ids)} documentos inseridos com sucesso")


AttributeError: 'NoneType' object has no attribute 'setCallSite'

In [26]:
spark.stop()

**MAPA COM GEOCODIFICAÇÃO**

In [27]:
!pip install folium pymongo geopy > /dev/null


[notice] A new release of pip is available: 24.3.1 -> 26.0.1
[notice] To update, run: pip install --upgrade pip


In [28]:
from pymongo import MongoClient
import folium
from geopy.geocoders import Nominatim
from time import sleep

# 🔌 MongoDB
MONGO_URI = os.environ.get("MONGO_URI")
DB_NAME = "etl_ips"
COLLECTION_NAME = "cinsscore_ips"

client = MongoClient(MONGO_URI)
db = client[DB_NAME]
collection = db[COLLECTION_NAME]

docs = list(collection.find({
    "city": {"$ne": ""},
    "country": {"$ne": ""}
}))

print(f"📊 Registros com cidade/país: {len(docs)}")

# 🌍 Geocoder (OpenStreetMap)
geolocator = Nominatim(user_agent="ip-map-colab")

# 🗺️ Mapa base (mundo)
mapa = folium.Map(
    location=[20, 0],
    zoom_start=2,
    tiles="CartoDB positron"
)

for d in docs:
    city = d.get("city")
    country = d.get("country")

    try:
        location = geolocator.geocode(f"{city}, {country}", timeout=10)
        sleep(1)  # evita bloqueio do serviço

        if not location:
            continue

        lat, lon = location.latitude, location.longitude

        popup = f"""
        <b>IP:</b> {d.get('listIP')}<br>
        <b>Cidade:</b> {city}<br>
        <b>País:</b> {country}<br>
        <b>Score:</b> {d.get('abuseConfidenceScore', 0)}<br>
        <b>Usuários distintos:</b> {d.get('numDistinctUsers', 0)}<br>
        <b>Total Reports:</b> {d.get('totalReports', 0)}
        """

        folium.CircleMarker(
            location=[lat, lon],
            radius=6 + min(d.get("totalReports", 0) / 20, 10),
            popup=popup,
            color="red" if d.get("totalReports", 0) > 0 else "blue",
            fill=True,
            fill_opacity=0.7
        ).add_to(mapa)

    except Exception as e:
        continue

# 🖥️ Exibir
mapa


📊 Registros com cidade/país: 200
